# n-gram 언어 모델링

In [1]:
import nltk
from nltk.util import ngrams
from collections import Counter

In [2]:
text = "오늘은 날씨가 좋다. 오늘은 기분이 좋다. 오늘은 일이 많다. 오늘은 사람이 많다. 오늘은 날씨가 맑다."

In [3]:
tokens = nltk.word_tokenize(text)

In [4]:
# 1-gram, 2-gram
unigram = tokens
bigram = list(ngrams(tokens, 2))

unigram_freq = Counter(unigram)
bigram_freq = Counter(bigram)
bigram_freq

Counter({('.', '오늘은'): 4,
         ('오늘은', '날씨가'): 2,
         ('좋다', '.'): 2,
         ('많다', '.'): 2,
         ('날씨가', '좋다'): 1,
         ('오늘은', '기분이'): 1,
         ('기분이', '좋다'): 1,
         ('오늘은', '일이'): 1,
         ('일이', '많다'): 1,
         ('오늘은', '사람이'): 1,
         ('사람이', '많다'): 1,
         ('날씨가', '맑다'): 1,
         ('맑다', '.'): 1})

In [5]:
# 조건부 확률 계산
for (w1, w2), freq in bigram_freq.items():
    prob = freq / unigram_freq[w1]
    print(f'P({w2}|{w1} = {prob:.3f})')

P(날씨가|오늘은 = 0.400)
P(좋다|날씨가 = 0.500)
P(.|좋다 = 1.000)
P(오늘은|. = 0.800)
P(기분이|오늘은 = 0.200)
P(좋다|기분이 = 1.000)
P(일이|오늘은 = 0.200)
P(많다|일이 = 1.000)
P(.|많다 = 1.000)
P(사람이|오늘은 = 0.200)
P(많다|사람이 = 1.000)
P(맑다|날씨가 = 0.500)
P(.|맑다 = 1.000)


### Perplexity 계산

In [6]:
import math

def compute_bigram_perplexity(test_text, unigram_freq, bigram_freq):
    test_tokens = nltk.word_tokenize(test_text)
    test_bigrams = list(ngrams(test_tokens, 2))

    log_prob_sum = 0
    N = len(test_bigrams)

    for bigram in test_bigrams:
        w1, w2 = bigram
        prob = bigram_freq.get(bigram, 0) / unigram_freq.get(w1, 1)
        if prob == 0:
            prob = 1e-10
        log_prob_sum += math.log2(prob)

    cross_entropy = -log_prob_sum / N
    perplexity = math.pow(2, cross_entropy)
    return perplexity

In [7]:
train_text = "자연어 처리는 재미있다. 자연어 처리는 어렵지만 도전하고 싶다. 오늘은 날씨가 좋다."

train_tokens = nltk.word_tokenize(train_text)

unigram = train_tokens
bigram = list(ngrams(train_tokens, 2))

unigram_freq = Counter(unigram)
bigram_freq = Counter(bigram)

In [8]:
test_sentences = [
    "자연어 처리는 재미있다.",
    "자연어 처리는 어렵지만 도전하고 싶다.",
    "오늘은 날씨가 좋다",
    "기계 번역은 어렵다.",
    "자연어 처리에 도전하고 싶다.",
    "오늘 날씨가 흐리다"
]


for sentence in test_sentences:
    pp = compute_bigram_perplexity(sentence, unigram_freq, bigram_freq)
    print(f'문장: {sentence} | perplexity: {pp:.4f}')

문장: 자연어 처리는 재미있다. | perplexity: 1.2599
문장: 자연어 처리는 어렵지만 도전하고 싶다. | perplexity: 1.1487
문장: 오늘은 날씨가 좋다 | perplexity: 1.0000
문장: 기계 번역은 어렵다. | perplexity: 10000000000.0000
문장: 자연어 처리에 도전하고 싶다. | perplexity: 100000.0000
문장: 오늘 날씨가 흐리다 | perplexity: 10000000000.0000
